Numba 0.50.0 Release Demo
=======================

This notebook contains a demonstration of new features present in the 0.50.0 release of Numba. Whilst release notes are produced as part of the [`CHANGE_LOG`](https://github.com/numba/numba/blob/release0.50/CHANGE_LOG), there's nothing like seeing code in action! This release contains a few new features, but it's mainly internals that have changed!

Key internal changes:

* Numba is now backed by LLVM 9.
* NumPy 1.18 support is added.


Demonstrations of new features/changes include:
* [New error messages](#New-error-messages)
* [Jitclass ``@staticmethod``](#Jitclass-@staticmethod-support)
* [NumPy enhancements](#Newly-supported-NumPy-functions/features)
* [All predicates are booleans!](#All-predicates-are-boolean!)

First, import the necessary from Numba and NumPy...

In [ ]:
from numba import jit, njit, config, __version__, errors
from numba.extending import overload
import numba
import numpy as np
assert tuple(int(x) for x in __version__.split('.')[:2]) >= (0, 50)

New error messages
=================

Providing error messages about any compilation issues Numba encounters is quite challenging, in Numba 0.50 the first of two important changes have been made to error reporting. In this change error messages have been made more compact, are more specific if information is available, and some exceptions that were being hidden are now correctly handled. Quick example:

In [ ]:
@njit
def illegal():
    return 2 + 'a'

try:
    illegal()
except Exception as e:
    print(e)

Jitclass `@staticmethod` support
===========================

In 0.50.0 a member of the Numba community (with thanks!) added ``@staticmethod`` support for Numba's experimental ``jitclass`` feature, an example:

In [ ]:
import numpy as np
from numba import int32, float32    # import the types
from numba.experimental import jitclass

spec = [
    ('value', int32),               # a simple scalar field
    ('array', float32[:]),          # an array field
]

@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)

    @property
    def size(self):
        return self.array.size

    def increment(self, val):
        for i in range(self.size):
            self.array[i] += val
        return self.array

    @staticmethod
    def add(x, y):
        return x + y

n = 21
mybag = Bag(n)
print(Bag.add(4, 5))
print(mybag.add(4.5, 5.6))
print(Bag.add.signatures)

Newly supported NumPy functions/features
=====
This release contains support for `np.sum` on the `timedelta64` type, againm this written by a member of the Numba community (with thanks!). NumPy 1.18 support is also implemented.

In [ ]:
a = np.arange(20).astype(np.dtype('timedelta64[M]'))

@njit
def demo_numpy(x):
    return np.sum(x)
    
demo_numpy(a)

All predicates are boolean!
======================

A long standing (literally years) bug was finally fixed which means that this common mistake of using a NumPy array as a predicate is at last reported correctly!

In [ ]:
@njit
def demo_predicates_are_bools():
    a = np.ones(5)
    if a:
        return 1
    else:
        return 0

try:
    demo_predicates_are_bools()
except ValueError as e:
    print(e)    